#### 한국어 코퍼스 전처리
- 데이터셋 : Korpora에서 로딩
- 형태소분석기 설정
- 단어사전 생성

[1] 모듈 로딩 및 데이터 준비 <hr>

In [3]:
## 모듈 로딩
from Korpora import Korpora                          # Open Korean Dataset
from konlpy.tag import *                             # 형태소 분석기
import spacy                                         # 형태소 분석기

from nltk.tokenize import word_tokenize, sent_tokenize, wordpunct_tokenize      # Pytorch 자연어처리 모듈

from torch.utils.data import Dataset, DataLoader     # Pytorch Dataset관련 모듈

import pandas as pd
import numpy as np

import re

In [4]:
## 데이터 로딩
nsmc = Korpora.load("nsmc")


    Korpora 는 다른 분들이 연구 목적으로 공유해주신 말뭉치들을
    손쉽게 다운로드, 사용할 수 있는 기능만을 제공합니다.

    말뭉치들을 공유해 주신 분들에게 감사드리며, 각 말뭉치 별 설명과 라이센스를 공유 드립니다.
    해당 말뭉치에 대해 자세히 알고 싶으신 분은 아래의 description 을 참고,
    해당 말뭉치를 연구/상용의 목적으로 이용하실 때에는 아래의 라이센스를 참고해 주시기 바랍니다.

    # Description
    Author : e9t@github
    Repository : https://github.com/e9t/nsmc
    References : www.lucypark.kr/docs/2015-pyconkr/#39

    Naver sentiment movie corpus v1.0
    This is a movie review dataset in the Korean language.
    Reviews were scraped from Naver Movies.

    The dataset construction is based on the method noted in
    [Large movie review dataset][^1] from Maas et al., 2011.

    [^1]: http://ai.stanford.edu/~amaas/data/sentiment/

    # License
    CC0 1.0 Universal (CC0 1.0) Public Domain Dedication
    Details in https://creativecommons.org/publicdomain/zero/1.0/

[Korpora] Corpus `nsmc` is already installed at C:\Users\KDP-14\Korpora\nsmc\ratings_train.txt
[Korpora] Corpus `nsmc` is already installed at C:\Users\KD

In [5]:
nsmc.train[0], nsmc.test[0]

(LabeledSentence(text='아 더빙.. 진짜 짜증나네요 목소리', label=0),
 LabeledSentence(text='굳 ㅋ', label=1))

In [6]:
## Test 데이터셋을 Dataframe으로 로딩
nsmcDF = pd.DataFrame(nsmc.test)
nsmcDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    50000 non-null  object
 1   label   50000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 781.4+ KB


In [7]:
### 단어 사전을 생성시에 활용
class TextDataset(Dataset):
    def __init__(self,feature,label):
        self.feature = feature
        self.label = label
        self.length = feature.shape[0]
    def __len__(self):
        return self.length
    
    def __getitem__(self,index):
        return self.feature[index], self.label[index]

## NSMC의 데이터셋 인스턴스 생성
nsmcDS = TextDataset(nsmcDF['text'],nsmcDF['label'])



In [8]:
nsmcDF.head(5)

,text,label
0,굳 ㅋ,1
1,GDNTOPCLASSINTHECLUB,0
2,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0


#### 한국어만 남기기 <hr>

In [9]:
def clean(text):
    result = re.compile('[^ㄱ-ㅎ가-힣]+')
    hangul = result.sub(' ',text)
    return hangul

nsmcDF['clean_hangul'] = nsmcDF.apply(lambda row: clean(row['text']),axis=1)

In [10]:
nsmcDF.head(5)

,text,label,clean_hangul
0,굳 ㅋ,1,굳 ㅋ
1,GDNTOPCLASSINTHECLUB,0,
2,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0,뭐야 이 평점들은 나쁘진 않지만 점 짜리는 더더욱 아니잖아
3,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0,지루하지는 않은데 완전 막장임 돈주고 보기에는
4,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0,만 아니었어도 별 다섯 개 줬을텐데 왜 로 나와서 제 심기를 불편하게 하죠


In [11]:
# 명사만 추출
from konlpy.tag import Okt

okt = Okt()

nsmcDF['token'] = nsmcDF.apply(lambda row: okt.nouns(row['clean_hangul']), axis=1)

#### 불용어 list
- 인터넷에 한국어 불용어 list 다운 + 불용어 list
<hr>

In [12]:
korea_stopword = r'C:\Users\KDP-14\Desktop\VSCode\KDT6\Torch_NLP\DAY10_07\stopwords.txt'

In [13]:
with open(korea_stopword, 'r', encoding='utf-8') as file:
    lines = file.readlines()

# 리스트 형태로 저장하기
lines = [line.strip() for line in lines]

In [14]:
type(lines)

list

- 불용어 & 구두점 제거

In [15]:
## 불용어 추출
from nltk.corpus import stopwords
import string

eng_stopwords = stopwords.words('english') # 리스트 형태
pun = list(string.punctuation) # 구두점 => 문자열 형태임 => 리스트로 형변환

In [16]:
type(pun)

list

In [17]:
stopwords = lines + pun
stopwords

['가',
 '가까스로',
 '가령',
 '각',
 '각각',
 '각자',
 '각종',
 '갖고말하자면',
 '같다',
 '같이',
 '개의치않고',
 '거니와',
 '거바',
 '거의',
 '것',
 '것과 같이',
 '것들',
 '게다가',
 '게우다',
 '겨우',
 '견지에서',
 '결과에 이르다',
 '결국',
 '결론을 낼 수 있다',
 '겸사겸사',
 '고려하면',
 '고로',
 '곧',
 '공동으로',
 '과',
 '과연',
 '관계가 있다',
 '관계없이',
 '관련이 있다',
 '관하여',
 '관한',
 '관해서는',
 '구',
 '구체적으로',
 '구토하다',
 '그',
 '그들',
 '그때',
 '그래',
 '그래도',
 '그래서',
 '그러나',
 '그러니',
 '그러니까',
 '그러면',
 '그러므로',
 '그러한즉',
 '그런 까닭에',
 '그런데',
 '그런즉',
 '그럼',
 '그럼에도 불구하고',
 '그렇게 함으로써',
 '그렇지',
 '그렇지 않다면',
 '그렇지 않으면',
 '그렇지만',
 '그렇지않으면',
 '그리고',
 '그리하여',
 '그만이다',
 '그에 따르는',
 '그위에',
 '그저',
 '그중에서',
 '그치지 않다',
 '근거로',
 '근거하여',
 '기대여',
 '기점으로',
 '기준으로',
 '기타',
 '까닭으로',
 '까악',
 '까지',
 '까지 미치다',
 '까지도',
 '꽈당',
 '끙끙',
 '끼익',
 '나',
 '나머지는',
 '남들',
 '남짓',
 '너',
 '너희',
 '너희들',
 '네',
 '넷',
 '년',
 '논하지 않다',
 '놀라다',
 '누가 알겠는가',
 '누구',
 '다른',
 '다른 방면으로',
 '다만',
 '다섯',
 '다소',
 '다수',
 '다시 말하자면',
 '다시말하면',
 '다음',
 '다음에',
 '다음으로',
 '단지',
 '답다',
 '당신',
 '당장',
 '대로 하다',
 '대하면',
 '대하여',
 '대해 말하자면',
 '대해서',
 '댕그',


In [18]:
def remove_stopwords(tokens):
    return [token for token in tokens if token not in stopwords]

In [19]:
nsmcDF['token'] = nsmcDF['token'].apply(lambda tokens: remove_stopwords(tokens))

In [20]:
nsmcDF

,text,label,clean_hangul,token
0,굳 ㅋ,1,굳 ㅋ,[]
1,GDNTOPCLASSINTHECLUB,0,,[]
2,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0,뭐야 이 평점들은 나쁘진 않지만 점 짜리는 더더욱 아니잖아,"[뭐, 평점, 점, 리, 더, 더욱]"
3,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0,지루하지는 않은데 완전 막장임 돈주고 보기에는,"[완전, 막장, 임, 돈, 보기]"
4,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0,만 아니었어도 별 다섯 개 줬을텐데 왜 로 나와서 제 심기를 불편하게 하죠,"[만, 별, 개, 심기]"
...,...,...,...,...
49995,오랜만에 평점 로긴했네ㅋㅋ 킹왕짱 쌈뽕한 영화를 만났습니다 강렬하게 육쾌함,1,오랜만에 평점 로긴했네ㅋㅋ 킹왕짱 쌈뽕한 영화를 만났습니다 강렬하게 육쾌함,"[만, 평점, 킹왕짱, 쌈뽕, 영화, 쾌함]"
49996,의지 박약들이나 하는거다 탈영은 일단 주인공 김대희 닮았고 이등병 찐따 OOOO,0,의지 박약들이나 하는거다 탈영은 일단 주인공 김대희 닮았고 이등병 찐따,"[의지, 박약, 탈영, 주인공, 김대희, 이등병]"
49997,그림도 좋고 완성도도 높았지만... 보는 내내 불안하게 만든다,0,그림도 좋고 완성도도 높았지만 보는 내내 불안하게 만든다,"[그림, 완성, 도도, 내내]"
49998,절대 봐서는 안 될 영화.. 재미도 없고 기분만 잡치고.. 한 세트장에서 다 해먹네,0,절대 봐서는 안 될 영화 재미도 없고 기분만 잡치고 한 세트장에서 다 해먹네,"[절대, 안, 영화, 재미, 기분, 잡, 세트, 해먹]"


- 토큰 숫자로  <hr>

In [21]:
from sklearn.feature_extraction.text import CountVectorizer

In [22]:
# CountVectorizer 객체 생성
vectorizer = CountVectorizer()

In [23]:
# 토큰 공백으로 붙여서 형태 변환 진행
nsmcDF['token_joined'] = nsmcDF['token'].apply(lambda tokens: ' '.join(tokens))

In [24]:
nsmcDF

,text,label,clean_hangul,token,token_joined
0,굳 ㅋ,1,굳 ㅋ,[],
1,GDNTOPCLASSINTHECLUB,0,,[],
2,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0,뭐야 이 평점들은 나쁘진 않지만 점 짜리는 더더욱 아니잖아,"[뭐, 평점, 점, 리, 더, 더욱]",뭐 평점 점 리 더 더욱
3,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0,지루하지는 않은데 완전 막장임 돈주고 보기에는,"[완전, 막장, 임, 돈, 보기]",완전 막장 임 돈 보기
4,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0,만 아니었어도 별 다섯 개 줬을텐데 왜 로 나와서 제 심기를 불편하게 하죠,"[만, 별, 개, 심기]",만 별 개 심기
...,...,...,...,...,...
49995,오랜만에 평점 로긴했네ㅋㅋ 킹왕짱 쌈뽕한 영화를 만났습니다 강렬하게 육쾌함,1,오랜만에 평점 로긴했네ㅋㅋ 킹왕짱 쌈뽕한 영화를 만났습니다 강렬하게 육쾌함,"[만, 평점, 킹왕짱, 쌈뽕, 영화, 쾌함]",만 평점 킹왕짱 쌈뽕 영화 쾌함
49996,의지 박약들이나 하는거다 탈영은 일단 주인공 김대희 닮았고 이등병 찐따 OOOO,0,의지 박약들이나 하는거다 탈영은 일단 주인공 김대희 닮았고 이등병 찐따,"[의지, 박약, 탈영, 주인공, 김대희, 이등병]",의지 박약 탈영 주인공 김대희 이등병
49997,그림도 좋고 완성도도 높았지만... 보는 내내 불안하게 만든다,0,그림도 좋고 완성도도 높았지만 보는 내내 불안하게 만든다,"[그림, 완성, 도도, 내내]",그림 완성 도도 내내
49998,절대 봐서는 안 될 영화.. 재미도 없고 기분만 잡치고.. 한 세트장에서 다 해먹네,0,절대 봐서는 안 될 영화 재미도 없고 기분만 잡치고 한 세트장에서 다 해먹네,"[절대, 안, 영화, 재미, 기분, 잡, 세트, 해먹]",절대 안 영화 재미 기분 잡 세트 해먹


token화된 상태보니까 영 이상함

In [25]:
# 문서의 단어를 숫자로 변환
X = vectorizer.fit_transform(nsmcDF['token_joined'])

In [26]:
X_df = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())

In [27]:
X_df

,가가,가각,가감,가게,가격,가계,가공,가관,가교,가구,...,힘들엇구,힘일,힘줄,힙겨웠다,힙스터,힙합,힛걸,힛쳐,힛츄,힛힛
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
49996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
49997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
49998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


* 딕셔너리 형태

In [28]:
sorted(vectorizer.vocabulary_.items(), reverse=False)

[('가가', 0),
 ('가각', 1),
 ('가감', 2),
 ('가게', 3),
 ('가격', 4),
 ('가계', 5),
 ('가공', 6),
 ('가관', 7),
 ('가교', 8),
 ('가구', 9),
 ('가구야', 10),
 ('가극', 11),
 ('가기', 12),
 ('가까이', 13),
 ('가끔', 14),
 ('가나', 15),
 ('가나다', 16),
 ('가나다라', 17),
 ('가난', 18),
 ('가년', 19),
 ('가늠', 20),
 ('가능', 21),
 ('가능성', 22),
 ('가다가', 23),
 ('가닥', 24),
 ('가당', 25),
 ('가대', 26),
 ('가도', 27),
 ('가두', 28),
 ('가득', 29),
 ('가든', 30),
 ('가디', 31),
 ('가디언', 32),
 ('가디언즈', 33),
 ('가뜩이나', 34),
 ('가라', 35),
 ('가라데', 36),
 ('가라이', 37),
 ('가람이', 38),
 ('가랑비', 39),
 ('가랑이', 40),
 ('가래', 41),
 ('가래침', 42),
 ('가랭', 43),
 ('가량', 44),
 ('가레스', 45),
 ('가레키', 46),
 ('가렐', 47),
 ('가렛', 48),
 ('가려진', 49),
 ('가로', 50),
 ('가루', 51),
 ('가루지기', 52),
 ('가르시아', 53),
 ('가르침', 54),
 ('가리봉동', 55),
 ('가마', 56),
 ('가마치', 57),
 ('가망', 58),
 ('가면', 59),
 ('가면라이더', 60),
 ('가무', 61),
 ('가문', 62),
 ('가물', 63),
 ('가뭄', 64),
 ('가미', 65),
 ('가바', 66),
 ('가발', 67),
 ('가방', 68),
 ('가버렷', 69),
 ('가보', 70),
 ('가볼때', 71),
 ('가부장', 72),
 ('가부장제', 73),
 ('가분', 74)